In [1]:
import pandas as pd
from sklearn import preprocessing
import tensorflow as tf
# display all columns
pd.options.display.max_columns = None

In [2]:
# Utils
from sklearn import preprocessing
def encode_text_index(df, name):
    le = preprocessing.LabelEncoder()
    df[name] = le.fit_transform(df[name])
    return le.classes_
# Encode a numeric column as zscores
def encode_numeric_zscore(df, name, mean=None, sd=None, test: bool = False, test_dataset = None):
    if test == True:
        if mean is None:
            mean = df[name].mean()
        if sd is None:
            sd = df[name].std()
        test_dataset[name] = (test_dataset[name] - mean) / sd
    else:
        if mean is None:
            mean = df[name].mean()
        if sd is None:
            sd = df[name].std()
        df[name] = (df[name] - mean) / sd

# Encode text values to dummy variables(i.e. [1,0,0],[0,1,0],[0,0,1] for red,green,blue)
def encode_text_dummy(df, name):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = f"{name}-{x}"
        df[dummy_name] = dummies[x]
    df.drop(name, axis=1, inplace=True)
# Convert a Pandas dataframe to the x,y inputs that TensorFlow needs
def to_xy(df, target):
    result = []
    for x in df.columns:
        if x != target:
            result.append(x)
    # find out the type of the target column.  Is it really this hard? :(
    target_type = df[target].dtypes
    target_type = target_type[0] if hasattr(
        target_type, '__iter__') else target_type
    # Encode to int for classification, float otherwise. TensorFlow likes 32 bits.
    if target_type in (np.int64, np.int32):
        # Classification
        dummies = pd.get_dummies(df[target])
        return df[result].values.astype(np.float32), dummies.values.astype(np.float32)
    # Regression
    return df[result].values.astype(np.float32), df[[target]].values.astype(np.float32)

In [3]:
path = "/Users/nguyenquangminh/Desktop/Thesis/Dataset/TON_IOT/Train_Test_datasets/Train_Test_Network_dataset/Train_Test_Network.csv"
dataset = pd.read_csv(filepath_or_buffer=path)
dataset

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,dst_ip_bytes,dns_query,dns_qclass,dns_qtype,dns_rcode,dns_AA,dns_RD,dns_RA,dns_rejected,ssl_version,ssl_cipher,ssl_resumed,ssl_established,ssl_subject,ssl_issuer,http_trans_depth,http_method,http_uri,http_version,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1554198358,3.122.49.24,1883,192.168.1.152,52976,tcp,-,80549.530260,1762852,41933215,OTH,0,252181,14911156,2,236,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,1554198358,192.168.1.79,47260,192.168.1.255,15600,udp,-,0.000000,0,0,S0,0,1,63,0,0,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,-,-,-,0,normal
2,1554198359,192.168.1.152,1880,192.168.1.152,51782,tcp,-,0.000000,0,0,OTH,0,0,0,0,0,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,1554198359,192.168.1.152,34296,192.168.1.152,10502,tcp,-,0.000000,0,0,OTH,0,0,0,0,0,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,-,-,-,0,normal
4,1554198362,192.168.1.152,46608,192.168.1.190,53,udp,dns,0.000549,0,298,SHR,0,0,0,2,354,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,1556340862,192.168.1.32,33108,176.28.50.165,80,tcp,-,0.000000,0,0,S0,0,1,60,0,0,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,-,-,-,1,xss
461039,1556423390,192.168.1.33,37242,34.230.157.88,443,tcp,-,0.000000,0,0,OTH,0,0,0,1,103,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,-,-,-,1,ransomware
461040,1556436603,192.168.1.37,4444,192.168.1.193,49178,tcp,-,290.371539,101568,2592,OTH,0,108,108064,31,3832,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,-,-,-,1,backdoor
461041,1556540442,192.168.1.31,60816,104.98.29.100,443,tcp,-,23.190902,32,31,SF,0,8,411,7,395,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,-,-,-,1,mitm


In [4]:
dataset.columns

Index(['ts', 'src_ip', 'src_port', 'dst_ip', 'dst_port', 'proto', 'service',
       'duration', 'src_bytes', 'dst_bytes', 'conn_state', 'missed_bytes',
       'src_pkts', 'src_ip_bytes', 'dst_pkts', 'dst_ip_bytes', 'dns_query',
       'dns_qclass', 'dns_qtype', 'dns_rcode', 'dns_AA', 'dns_RD', 'dns_RA',
       'dns_rejected', 'ssl_version', 'ssl_cipher', 'ssl_resumed',
       'ssl_established', 'ssl_subject', 'ssl_issuer', 'http_trans_depth',
       'http_method', 'http_uri', 'http_version', 'http_request_body_len',
       'http_response_body_len', 'http_status_code', 'http_user_agent',
       'http_orig_mime_types', 'http_resp_mime_types', 'weird_name',
       'weird_addl', 'weird_notice', 'label', 'type'],
      dtype='object')

In [5]:
# Dropping columns
dataset.drop(columns=["ts", "src_ip", "dst_ip"], inplace=True)

In [6]:
dataset

,src_port,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,dst_ip_bytes,dns_query,dns_qclass,dns_qtype,dns_rcode,dns_AA,dns_RD,dns_RA,dns_rejected,ssl_version,ssl_cipher,ssl_resumed,ssl_established,ssl_subject,ssl_issuer,http_trans_depth,http_method,http_uri,http_version,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type
0,1883,52976,tcp,-,80549.530260,1762852,41933215,OTH,0,252181,14911156,2,236,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
1,47260,15600,udp,-,0.000000,0,0,S0,0,1,63,0,0,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,-,-,-,0,normal
2,1880,51782,tcp,-,0.000000,0,0,OTH,0,0,0,0,0,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,bad_TCP_checksum,-,F,0,normal
3,34296,10502,tcp,-,0.000000,0,0,OTH,0,0,0,0,0,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,-,-,-,0,normal
4,46608,53,udp,dns,0.000549,0,298,SHR,0,0,0,2,354,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,bad_UDP_checksum,-,F,0,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
461038,33108,80,tcp,-,0.000000,0,0,S0,0,1,60,0,0,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,-,-,-,1,xss
461039,37242,443,tcp,-,0.000000,0,0,OTH,0,0,0,1,103,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,-,-,-,1,ransomware
461040,4444,49178,tcp,-,290.371539,101568,2592,OTH,0,108,108064,31,3832,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,-,-,-,1,backdoor
461041,60816,443,tcp,-,23.190902,32,31,SF,0,8,411,7,395,-,0,0,0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0,0,0,-,-,-,-,-,-,1,mitm


In [7]:
dataset["service"].unique()

array(['-', 'dns', 'ssl', 'dhcp', 'http', 'ftp', 'dce_rpc', 'gssapi',
       'smb;gssapi', 'smb'], dtype=object)

In [8]:
dataset["service"].describe()

count     461043
unique        10
top            -
freq      280216
Name: service, dtype: object

In [9]:
for column in dataset.columns:
    print(column, dataset[column].dtype)

src_port int64
dst_port int64
proto object
service object
duration float64
src_bytes int64
dst_bytes int64
conn_state object
missed_bytes int64
src_pkts int64
src_ip_bytes int64
dst_pkts int64
dst_ip_bytes int64
dns_query object
dns_qclass int64
dns_qtype int64
dns_rcode int64
dns_AA object
dns_RD object
dns_RA object
dns_rejected object
ssl_version object
ssl_cipher object
ssl_resumed object
ssl_established object
ssl_subject object
ssl_issuer object
http_trans_depth object
http_method object
http_uri object
http_version object
http_request_body_len int64
http_response_body_len int64
http_status_code int64
http_user_agent object
http_orig_mime_types object
http_resp_mime_types object
weird_name object
weird_addl object
weird_notice object
label int64
type object


In [10]:
print(dataset["dns_query"].value_counts())
dataset.drop(columns=["dns_query"], inplace=True)

-                                                  366019
a2z3kk2ebqzso7.iot.ap-southeast-2.amazonaws.com     13700
arpa.whois-servers.net                               2198
_sleep-proxy._udp.local                              2187
testphp.vulnweb.com                                  1848
                                                    ...  
94.157.81.140.in-addr.arpa                              1
125.48.34.73.in-addr.arpa                               1
101.192.178.247.in-addr.arpa                            1
254.68.212.in-addr.arpa                                 1
loadm.exelator.com                                      1
Name: dns_query, Length: 14149, dtype: int64


In [11]:
print(dataset["conn_state"].value_counts())

SF        123512
S0        113495
OTH       111842
REJ        45036
SHR        22053
SH         18138
S1         13843
S3          6642
RSTR        2360
RSTRH       1724
RSTO        1518
S2           708
RSTOS0       172
Name: conn_state, dtype: int64


In [12]:
print(dataset["dns_AA"].value_counts())

-    365158
F     81279
T     14606
Name: dns_AA, dtype: int64


In [13]:
print(dataset["dns_RD"].value_counts())

-    365158
F     63106
T     32779
Name: dns_RD, dtype: int64


In [14]:
print(dataset["dns_rejected"].value_counts())

-    365158
F     89392
T      6493
Name: dns_rejected, dtype: int64


In [15]:
print(dataset["ssl_version"].value_counts())

-         460737
TLSv12       188
TLSv13       116
TLSv10         2
Name: ssl_version, dtype: int64


In [16]:
print(dataset["ssl_cipher"].value_counts())

-                                          460737
TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256         176
TLS_AES_128_GCM_SHA256                        116
TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384          11
TLS_ECDHE_RSA_WITH_AES_256_CBC_SHA              2
TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256         1
Name: ssl_cipher, dtype: int64


In [17]:
print(dataset["ssl_resumed"].value_counts())

-    460352
F       605
T        86
Name: ssl_resumed, dtype: int64


In [18]:
print(dataset["ssl_established"].value_counts())

-    460352
F       596
T        95
Name: ssl_established, dtype: int64


In [19]:
print(dataset["ssl_subject"].value_counts())

-                                                                                           461034
CN=*.wns.windows.com                                                                             4
CN=Mr Shepherd;OU=Security Shepherd Project;O=OWASP;L=Dublin;ST=Ireland;C=IE                     2
CN=settings-win.data.microsoft.com;OU=WSE;O=Microsoft;L=Redmond;ST=WA;C=US                       1
CN=sls.update.microsoft.com;OU=DSP;O=Microsoft;L=Redmond;ST=WA;C=US                              1
CN=*.events.data.microsoft.com;OU=Microsoft;O=Microsoft Corporation;L=Redmond;ST=WA;C=US         1
Name: ssl_subject, dtype: int64


In [20]:
print(dataset["ssl_issuer"].value_counts())

-                                                                                                461034
CN=Microsoft IT TLS CA 5;OU=Microsoft IT;O=Microsoft Corporation;L=Redmond;ST=Washington;C=US         4
CN=Microsoft Secure Server CA 2011;O=Microsoft Corporation;L=Redmond;ST=Washington;C=US               2
CN=Mr Shepherd;OU=Security Shepherd Project;O=OWASP;L=Dublin;ST=Ireland;C=IE                          2
CN=Microsoft Update Secure Server CA 2.1;O=Microsoft Corporation;L=Redmond;ST=Washington;C=US         1
Name: ssl_issuer, dtype: int64


In [21]:
print(dataset["http_trans_depth"].value_counts())

-     460796
1        214
2         25
10         1
3          1
4          1
5          1
6          1
7          1
8          1
9          1
Name: http_trans_depth, dtype: int64


In [22]:
print(dataset["http_method"].value_counts())

-       460809
GET        215
POST        15
HEAD         4
Name: http_method, dtype: int64


In [23]:
print(dataset["http_uri"].value_counts())

-                                                                                                                                                                                         460809
/no_nonce_string/                                                                                                                                                                             15
/dvwa/login.php-r                                                                                                                                                                             13
/no_nonce_string/ContentDirectory/scpd.xml                                                                                                                                                    12
/filestreamingservice/files/1f41d40e-6e49-42f0-a4dc-3d35029e2a5d?P1=1554315773&P2=402&P3=2&P4=D2MtshBIyXICLz+f8ls5Sa4mWcxA1B670oj1WRainhnd91v77n9XO6/xoc4QeEYZdwdKSG+6cFNvkBVYbJdXpw==         9
                                   

In [24]:
print(dataset["http_version"].value_counts())

-      460801
1.1       242
Name: http_version, dtype: int64


In [25]:
print(dataset["http_user_agent"].value_counts())

-                                                                                                                                    460809
Microsoft-Delivery-Optimization/10.0                                                                                                     57
Microsoft-Windows/10.0 UPnP/1.0                                                                                                          32
Ruby                                                                                                                                     31
DAFUPnP                                                                                                                                  19
User-Agent: Microsoft-DLNA DLNADOC/1.50                                                                                                  19
Windows-Update-Agent/10.0.10011.16384 Client-Protocol/1.91                                                                               11
MICROSOFT_DEVICE_MET

In [26]:
print(dataset["http_orig_mime_types"].value_counts())

-                       461029
application/xml              8
application/soap+xml         6
Name: http_orig_mime_types, dtype: int64


In [27]:
print(dataset["http_resp_mime_types"].value_counts())

-                                    460883
application/xml                          63
text/html                                41
image/png                                15
application/vnd.ms-cab-compressed        13
text/plain                               11
application/ocsp-response                 8
text/json                                 4
image/jpeg                                4
application/x-debian-package              1
Name: http_resp_mime_types, dtype: int64


In [28]:
print(dataset["weird_name"].value_counts())

-                                   459749
DNS_RR_unknown_type                    753
active_connection_reuse                275
data_before_established                109
bad_UDP_checksum                        68
bad_TCP_checksum                        52
connection_originator_SYN_ack           20
above_hole_data_without_any_acks         6
inappropriate_FIN                        3
dnp3_corrupt_header_checksum             3
possible_split_routing                   3
TCP_ack_underflow_or_misorder            2
Name: weird_name, dtype: int64


In [29]:
print(dataset["weird_addl"].value_counts())

-     460290
46       707
48        38
43         8
Name: weird_addl, dtype: int64


In [30]:
print(dataset["weird_notice"].value_counts())

-    459749
F      1294
Name: weird_notice, dtype: int64


In [31]:
# Dropping type column
dataset.drop(columns=["label"], inplace=True)

In [32]:
for column in dataset.columns:
    if column != "type":
        if dataset[column].dtype == "object":
            encode_text_dummy(dataset, column)
        else:
            encode_numeric_zscore(dataset, column)

    else:
        print(encode_text_index(dataset, column))

['backdoor' 'ddos' 'dos' 'injection' 'mitm' 'normal' 'password'
 'ransomware' 'scanning' 'xss']


In [33]:
dataset

,src_port,dst_port,duration,src_bytes,dst_bytes,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,dst_ip_bytes,dns_qclass,dns_qtype,dns_rcode,http_request_body_len,http_response_body_len,http_status_code,type,proto-icmp,proto-tcp,proto-udp,service--,service-dce_rpc,service-dhcp,service-dns,service-ftp,service-gssapi,service-http,service-smb,service-smb;gssapi,service-ssl,conn_state-OTH,conn_state-REJ,conn_state-RSTO,conn_state-RSTOS0,conn_state-RSTR,conn_state-RSTRH,conn_state-S0,conn_state-S1,conn_state-S2,conn_state-S3,conn_state-SF,conn_state-SH,conn_state-SHR,dns_AA--,dns_AA-F,dns_AA-T,dns_RD--,dns_RD-F,dns_RD-T,dns_RA--,dns_RA-F,dns_RA-T,dns_rejected--,dns_rejected-F,dns_rejected-T,ssl_version--,ssl_version-TLSv10,ssl_version-TLSv12,ssl_version-TLSv13,ssl_cipher--,ssl_cipher-TLS_AES_128_GCM_SHA256,ssl_cipher-TLS_ECDHE_ECDSA_WITH_AES_128_GCM_SHA256,ssl_cipher-TLS_ECDHE_RSA_WITH_AES_128_GCM_SHA256,ssl_cipher-TLS_ECDHE_RSA_WITH_AES_256_CBC_SHA,ssl_cipher-TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384,ssl_resumed--,ssl_resumed-F,ssl_resumed-T,ssl_established--,ssl_established-F,ssl_established-T,ssl_subject--,ssl_subject-CN=*.events.data.microsoft.com;OU=Microsoft;O=Microsoft Corporation;L=Redmond;ST=WA;C=US,ssl_subject-CN=*.wns.windows.com,ssl_subject-CN=Mr Shepherd;OU=Security Shepherd Project;O=OWASP;L=Dublin;ST=Ireland;C=IE,ssl_subject-CN=settings-win.data.microsoft.com;OU=WSE;O=Microsoft;L=Redmond;ST=WA;C=US,ssl_subject-CN=sls.update.microsoft.com;OU=DSP;O=Microsoft;L=Redmond;ST=WA;C=US,ssl_issuer--,ssl_issuer-CN=Microsoft IT TLS CA 5;OU=Microsoft IT;O=Microsoft Corporation;L=Redmond;ST=Washington;C=US,ssl_issuer-CN=Microsoft Secure Server CA 2011;O=Microsoft Corporation;L=Redmond;ST=Washington;C=US,ssl_issuer-CN=Microsoft Update Secure Server CA 2.1;O=Microsoft Corporation;L=Redmond;ST=Washington;C=US,ssl_issuer-CN=Mr Shepherd;OU=Security Shepherd Project;O=OWASP;L=Dublin;ST=Ireland;C=IE,http_trans_depth--,http_trans_depth-1,http_trans_depth-10,http_trans_depth-2,http_trans_depth-3,http_trans_depth-4,http_trans_depth-5,http_trans_depth-6,http_trans_depth-7,http_trans_depth-8,http_trans_depth-9,http_method--,http_method-GET,http_method-HEAD,http_method-POST,http_uri--,http_uri-/,http_uri-/37;51;99;37;54;53;37;54;100;37;54;50;37;54;53;37;54;52;37;50;48;37;55;51;37;55;50;37;54;51;37;51;100;37;50;50;37;51;52;37;54;52;37;54;52;37;51;53;37;51;49;37;54;52;37;51;55;37;54;50;37;51;50;37;54;51;37;51;51;37;51;53;37;51;51;37;51;57;37;54;54;37;54;52;37;51;55;37;51;53;37;51;49;37;51;49;37;54;49;37;51;54;37;51;50;37;51;51;37;51;49;37;51;48;37;54;51;37;54;53;37;51;48;37;54;54;37;51;55;37;51;51;37;50;50;37;51;101,http_uri-/37;51;99;37;55;51;37;54;51;37;55;50;37;54;57;37;55;48;37;55;52;37;50;48;37;55;51;37;55;50;37;54;51;37;51;100;37;51;49;37;50;48;37;54;56;37;55;50;37;54;53;37;54;54;37;51;100;37;51;49;37;50;48;37;54;102;37;54;101;37;54;53;37;55;50;37;55;50;37;54;102;37;55;50;37;51;100;37;50;50;37;54;97;37;54;49;37;55;54;37;54;49;37;55;51;37;54;51;37;55;50;37;54;57;37;55;48;37;55;52;37;51;97;37;51;53;37;51;48;37;51;57;37;51;55;37;51;57;37;51;57;37;51;48;37;51;50;37;54;52;37;54;53;37;51;55;37;54;50;37;51;48;37;51;54;37;51;51;37;54;49;37;51;56;37;51;55;37;54;51;37;51;51;37;51;53;37;51;54;37;51;48;37;51;50;37;51;51;37;54;52;37;51;57;37;54;53;37;51;56;37;51;54;37;54;50;37;54;52;37;50;50;37;51;101;37;51;99;37;50;102;37;55;51;37;54;51;37;55;50;37;54;57;37;55;48;37;55;52;37;51;101,http_uri-/?C=D;O=A,http_uri-/?C=D;O=D,http_uri-/?C=M;O=D,http_uri-/?C=N;O=A,http_uri-/?C=S;O=A,http_uri-/?C=S;O=D,http_uri-/?id=3') UNION ALL SELECT NULL;NULL;NULL;NULL;NULL;NULL;NULL;NULL-- uzzZ,http_uri-/?id=3');SELECT PG_SLEEP(5)--,http_uri-/AVTransport_1.xml,http_uri-/ConnectionManager_1.xml,http_uri-/MFEwTzBNMEswSTAJBgUrDgMCGgUABBTBL0V27RVZ7LBduom/nYB45SPUEwQU5Z1ZMIJHWMys+ghUNoZ7OrUETfACEA8sEMlbBsCTf7jUSfg+hWk=,http_uri-/RenderingControl_1.xml,http_uri-/c/msdownload/update/others/2019/04/28698818_bfacd682bfb9bf10a7396fb5e080fa0a38d1b153.cab,http_uri-/c/msdownload/update/others/

In [34]:
import pandas as pd
import io
import requests
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.callbacks import EarlyStopping

# Break into X (predictors) & y (prediction)
x, y = to_xy(dataset,'type')

# Create a test/train split.  25% test
# Split into train/test
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, random_state=42)

# Create neural net
model = Sequential()
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(50, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(10, input_dim=x.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
model.add(Dense(y.shape[1],activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='auto')
model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=1000)

Epoch 1/1000
10806/10806 - 10s - loss: 0.7410 - val_loss: 0.6276 - 10s/epoch - 911us/step
Epoch 2/1000
10806/10806 - 9s - loss: 0.6022 - val_loss: 0.5956 - 9s/epoch - 847us/step
Epoch 3/1000
10806/10806 - 9s - loss: 0.5782 - val_loss: 0.5751 - 9s/epoch - 845us/step
Epoch 4/1000
10806/10806 - 9s - loss: 0.5669 - val_loss: 0.5595 - 9s/epoch - 844us/step
Epoch 5/1000
10806/10806 - 9s - loss: 0.5583 - val_loss: 0.5590 - 9s/epoch - 866us/step
Epoch 6/1000
10806/10806 - 9s - loss: 0.5532 - val_loss: 0.5884 - 9s/epoch - 840us/step
Epoch 7/1000
10806/10806 - 9s - loss: 0.5465 - val_loss: 0.5442 - 9s/epoch - 833us/step
Epoch 8/1000
10806/10806 - 9s - loss: 0.5429 - val_loss: 0.5956 - 9s/epoch - 839us/step
Epoch 9/1000
10806/10806 - 9s - loss: 0.5417 - val_loss: 0.5455 - 9s/epoch - 850us/step
Epoch 10/1000
10806/10806 - 10s - loss: 0.5339 - val_loss: 0.5441 - 10s/epoch - 899us/step
Epoch 11/1000
10806/10806 - 9s - loss: 0.5302 - val_loss: 0.5347 - 9s/epoch - 872us/step
Epoch 12/1000
10806/10806 

In [35]:
# Measure accuracy
pred = model.predict(x_test)
pred = np.argmax(pred,axis=1)
y_eval = np.argmax(y_test,axis=1)
score = metrics.accuracy_score(y_eval, pred)
print("Validation score: {}".format(score))

3602/3602 [==============================] - 2s 467us/step
Validation score: 0.8646723523134452
